#  Вычислительная устойчивость (черновик)

Под [вычислительной устойчивостью](https://ru.wikipedia.org/wiki/Вычислительная_устойчивость) понимают свойство алгоритма не увеличивать ошибку данных,
т.е. небольшие вариации входных данных должны приводить к небольшому изменению результата. 
Существует несколько точных математических формулировок устойчивости, некоторые из которых будут рассмотрены ниже.

В качестве примера рассмотрим задачу решения кубического уравнения 
([Nick Higham. Accuracy and Stability of Numerical Algorithms](https://www.maths.manchester.ac.uk/~higham/asna/index.php) 26.3.3. Roots of a Cubic).
Рассмотрим [кубическое уравнение](https://ru.wikipedia.org/wiki/Кубическое_уравнение) 
$$x^3+ax^2+bx+c=0$$ 
и приведем его к каноническому виду
$$y^3+py+q=0,\quad
p=b-\frac{a^2}3,\quad
q=\frac{2a^3}{27}-\frac{ab}{3}+c,$$ 
заменой $x=y-a/3$.
Замена Виета $y=w-p/(3w)$ приводит уравнение к бикубическому,
$$
w^3-\frac{p^3}{27w^3}+q=0\quad\Leftrightarrow\quad
(w^3)^2+qw^3-\frac{p^3}{27}=0.
$$
Решая квадратное относительно $w^3$ уравнение, получаем:
$$w^3=-\frac{q}{2}\pm\sqrt{\frac{q^2}{4}+\frac{p^3}{27}}.$$
Извлекая кубический корень находим три различных корня (причем достаточно взять любой один из знаков в формуле выше).
Делая обратные подстановки $w\mapsto y\mapsto x$, получаем искомые решения уравнения.

Таким образом мы получили явные аналитические формулы для расчета корней кубического уравнения,
полностью решив задачу с точки зрения математики.
Проверим, так ли хороши эти формулы для численного счета.
Для этого оценим устойчивость метода.

In [1]:
import numpy as np

In [2]:
# Определим функцию, решающую уравнение третьего порядка.
def solve_cubic(coefs):
    """
    Аргумент функции coefs=[a,b,c] задает уравнение x**3_a*x**2+b*x+c=0.
    Фнукция возвращает три корня этого уравнения [x0,x1,x2].
    """
    a, b, c = coefs
    p = b - a**2/3
    q = 2*a**3/27-a*b/3+c
    disc = q**2/4+p**3/27 # дискриминант бикубического уравнения
    w_cubed = -q/2+np.sqrt(disc+0j) 
    # Три кубических корня из w_cubed.
    rho, phi = np.abs(w_cubed), np.angle(w_cubed)
    ws = np.cbrt(rho)*np.exp(1j*(phi+np.array([0,2,-2])*np.pi)/3)   
    # Вспомогательная функция, преобразующая w в x.
    def w2x(w):
        y = w-p/(3*w)
        x = y-a/3
        return x
    return w2x(ws)

# Для примера решим уравнение x^3-2x^2-x+2,
with np.printoptions(precision=2, suppress=True):
    print( "Roots of x^3-2x^2-x+2=0:", solve_cubic([-2,-1,2]) )

Roots of x^3-2x^2-x+2=0: [ 2.-0.j -1.-0.j  1.+0.j]


In [3]:
# Выясним, насколько точные корни дает наша функция.
def relative_error(x, x0):
    """
    Считает относительную погрешность |x-x0|/|x0| в норме l-infty для                                                                                                                                                                                               .
    Так как порядок корней не фиксирован, то выбирается минимальная ошибка из всех перестановок.
    """
    x0 = np.asarray(x0)
    x = np.asarray(x)
    permutations = [ [0,1,2], [0,2,1], [1,2,0], [2,1,0], [2,0,1], [1,0,2] ]
    abserr = [ np.linalg.norm(x[p]-x0, ord=np.inf) for p in permutations]
    return np.min(abserr) / np.linalg.norm(x0)

# Простой тест на предыдущем многочлене
assert relative_error(solve_cubic([-2,-1,2]), [2,-1,1])<1e-15

# Для испытаний нам удобно иметь корни явно, тогда нам нужна функция для вычисления коэффициентов многочлена через корни.
def roots2coefs(roots):
    x0,x1,x2=roots
    return [-(x0+x1+x2), x0*x1+x1*x2+x2*x0, -x0*x1*x2]

# Снова проверяем на примере
import numpy.testing as npt
npt.assert_allclose( roots2coefs([2,-1,1]), [-2,-1,2] )
        
# Далее нам удобно собрать все части в одну функцию.
def test_cubic_solve(roots):
    """Функция возвращает ошибку вычисления функцией cubic_solve корней многочлена, заданного списком его корней."""
    err = relative_error( solve_cubic(roots2coefs(roots)), roots )
#     print(err)
    return err

# Небольшая проверка: считаем ошибку на случайном многочлене.
print( "Error on a random polynomial:", test_cubic_solve(np.random.randn(3)) )

# Ошибка на случайном многочлене типично мала. Однако равномерно ли ограниченна ошибка?
# Возьмем пример из Higham'а
print( "Error for Higham's example :", test_cubic_solve([-1.6026, -6.4678e-2 + 8.8798e-1j, -6.4678e-2 - 8.8798e-1j]) )
# Ошибка 2e-4 (у Higham 1e-2 на несколько отличных коэффициентах) на много порядков выше ошибки исходных данных.
# Что-то явно не так с нашим алгоритмом.

Error on a random polynomial: 1.616771613060855e-16
Error for Higham's example : 0.00019053197555237155


In [4]:
# Чтобы оценить предельную погрешность, не нужно искать примеры в литературе, можно численно максимизировать функционал ошибки.
from scipy.optimize import minimize
def find_worst_case():
    """Вычисляет многочлен, на котором ошибка вычисления solve_cubic максимальна."""
    def real_to_complex(r): # Преобразует тройку вещественных чисел в корни.
        return [r[0], r[1]+r[2]*1j, r[1]-r[2]*1j]
    roots = np.random.randn(3) # Начальный набор нулей многочлена.
    res = minimize(lambda r: -test_cubic_solve(real_to_complex(r)), roots, method='Nelder-Mead', options={"disp":True, "fatol":1e-16})
    return real_to_complex(res.x)

# Найдем наихудший многочлен, на котором ошибка больше всего.
roots = find_worst_case()
print(f"Error for roots {roots}: {test_cubic_solve(roots)} ")

Optimization terminated successfully.
         Current function value: -0.000000
         Iterations: 132
         Function evaluations: 338
Error for roots [0.17799910776935274, (0.45268929988808415-0.44600828164939554j), (0.45268929988808415+0.44600828164939554j)]: 4.411003433512207e-16 


In [5]:
# Так как функция находит только локальный максимум, то потребуется несколько запусков, чтобы получить большую ошибку.
# В какой-то момент мы получим сообщение о делении на ноль в функции cubic_solve, так как w=0.
# В этот момент ошибка становится бесконечно большой.
# Округляя корни, мы можем получить точку, в которой ошибка велика, но не бесконечна, например:
roots = [-1.66827, (-0.715961-0.54981j), (-0.715961+0.54981j)]
print("Cubic_solve error:", test_cubic_solve(roots) )
# Очевидно, что метод неустойчив, так как он неограниченно увеличивает ошибку в исходных данных.
# Однако возможно, что задача нахождения корней плохо обусловлена, тогда никакой метод не даст ответ с хорошей точностью.
# Из теории известно, что задача нахождения корней многочлена хорошо обусловлена, если корни многочлена отделены друг от друга, что выполняется в нашем случае.
# Мы можем убедиться в этом косвенно, проверив, что NumPy позволяет найти эти корни с гораздо большей точностью.
roots_numpy = np.roots([1]+roots2coefs(roots)) 
print("NumPy.roots error:", relative_error(roots_numpy, roots))
# Раз задача хорошо обусловлена, то значит наш конкретный способ счета неустойчив.
# Таким образом явные аналитические формулы не всегда дают лучший способ счета.

Cubic_solve error: 0.03429396544184734
NumPy.roots error: 2.2253698594795346e-15


## Задание.

1. Каким образом NumPy вычисляет корни многочлена? Почему этот способ лучше? 